In [7]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature = 0,
    groq_api_key = 'gsk_##################################################',
    model_name="llama-3.1-70b-versatile"
)

# response = llm.invoke("who was the first person on earth")
# print(response.content)

**webscraping job portal using web base loader**

In [ ]:
#!pip install langchain_community

In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-41553?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Software Engineer II- Frontend Developer, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer II- Frontend Developer, ITCKarnataka, IndiaBecome a P

In [9]:
from langchain_core.prompts import PromptTemplate 

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the following keys: `role`,`skills`,`experience`,`job_details`
    Only return the valid json.
    ### VALID JSON (NO PREAMBLE) :
    """
)
chain = prompt_extract | llm #chain in langchain
response = chain.invoke(input={'page_data':page_data})
print(response.content)

```json
{
  "role": "Software Engineer II- Frontend Developer, ITC",
  "skills": [
    "Java",
    "Spring Boot",
    "ReactJS",
    "HTML5",
    "CSS3",
    "JavaScript",
    "Typescript",
    "Scrum",
    "Agile",
    "UI/UX principles",
    "Continuous integration",
    "Unit testing",
    "Static analysis",
    "Automated integration tests",
    "Continuous delivery",
    "AWS Services",
    "GCE",
    "Rackspace",
    "Azure",
    "Micro service architecture",
    "Domain driven design",
    "RESTful Services",
    "Splunk",
    "SignalFx",
    "New Relic",
    "NoSQL data stores",
    "Cassandra",
    "DynamoDB",
    "MongoDB",
    "Relational data stores",
    "MySQL",
    "Oracle"
  ],
  "experience": [
    "Minimum 3-5 years software development experience",
    "Experience working in a highly collaborative, multi-discipline development team environment",
    "Experience working in a distributed/cloud-based environment",
    "Experience in designing and implementing highly sca

In [ ]:
type(response.content)

converting this str to an actual json object (dictionary)

In [10]:
from langchain_core.output_parsers import JsonOutputParser

json_res = JsonOutputParser().parse(response.content)
print(json_res)

{'role': 'Software Engineer II- Frontend Developer, ITC', 'skills': ['Java', 'Spring Boot', 'ReactJS', 'HTML5', 'CSS3', 'JavaScript', 'Typescript', 'Scrum', 'Agile', 'UI/UX principles', 'Continuous integration', 'Unit testing', 'Static analysis', 'Automated integration tests', 'Continuous delivery', 'AWS Services', 'GCE', 'Rackspace', 'Azure', 'Micro service architecture', 'Domain driven design', 'RESTful Services', 'Splunk', 'SignalFx', 'New Relic', 'NoSQL data stores', 'Cassandra', 'DynamoDB', 'MongoDB', 'Relational data stores', 'MySQL', 'Oracle'], 'experience': ['Minimum 3-5 years software development experience', 'Experience working in a highly collaborative, multi-discipline development team environment', 'Experience working in a distributed/cloud-based environment', 'Experience in designing and implementing highly scalable web applications', 'Experience with monitoring and tracing tools', 'Exposure to Supply Chain domain areas', 'Experience working with NoSQL data stores', 'Expe

In [ ]:
type(json_res)

In [11]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


**now we have to put in chromadb**

In [12]:
import chromadb
import uuid
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents = row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [12]:
links = collection.query(query_texts=["Experience in Python","Experience in React"], n_results=2).get("metadatas")
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [13]:
job = json_res['skills']
job

['Java',
 'Spring Boot',
 'ReactJS',
 'HTML5',
 'CSS3',
 'JavaScript',
 'Typescript',
 'Scrum',
 'Agile',
 'UI/UX principles',
 'Continuous integration',
 'Unit testing',
 'Static analysis',
 'Automated integration tests',
 'Continuous delivery',
 'AWS Services',
 'GCE',
 'Rackspace',
 'Azure',
 'Micro service architecture',
 'Domain driven design',
 'RESTful Services',
 'Splunk',
 'SignalFx',
 'New Relic',
 'NoSQL data stores',
 'Cassandra',
 'DynamoDB',
 'MongoDB',
 'Relational data stores',
 'MySQL',
 'Oracle']

In [14]:
links = collection.query(query_texts=job, n_results=2).get("metadatas")
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.

**next we generate a prompt for generating a cold email**

In [15]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Mohini, a business development executive at Tinfosys. Tinfosys is an AI & Software Company.
    the seamless integration of business processes through automated tools.
    Over our experience, we have empowered numerous enterprises with tailored solutions, process optimization,cost reduction, and heightened overall efficiency.
    Your job is to write a cold email to the client regarding the job mentioned above in fulfulling their needs.
    Also add the most relevant ones from the following links to showcase Tinfosys's portfolio.
    Remember you are Mohini, BDE at Tinfosys.
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    """
)

chain_email = prompt_email | llm
email = chain_email.invoke({"job_description":str(job),"link_list": links})
print(email.content)

Subject: Unlock Efficient Business Processes with Tinfosys's Expertise

Dear [Client's Name],

I came across your job description and was impressed by the cutting-edge technologies you're looking to implement. As a Business Development Executive at Tinfosys, I'd like to introduce you to our AI & Software Company that specializes in seamless integration of business processes through automated tools.

Our team of experts has extensive experience in designing and developing scalable solutions using Java, Spring Boot, ReactJS, and other technologies mentioned in your job description. We've successfully empowered numerous enterprises with tailored solutions, process optimization, cost reduction, and heightened overall efficiency.

Our expertise in Microservice Architecture, Domain-Driven Design, and RESTful Services can help you achieve a robust and efficient system. We're also well-versed in both NoSQL data stores (Cassandra, DynamoDB, MongoDB) and Relational data stores (MySQL, Oracle), e

**creating streamlit UI**